## Errores v1.1

In [20]:
%load_ext autoreload
%autoreload 2
import json
from mongoengine import connect
from tweepyrate import create_apps
from hatespeech_models import Tweet, Article

client = connect("hatespeech-news")

db = client["hatespeech-news"]

print(f"Tenemos {Article.objects.count()} artículos")

with open("../../dumps/coronavirus-argentina-v1.1.json") as f:
    arts_v11 = json.load(f)
    v11_ids = {int(art["tweet_id"]) for art in arts_v11}
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Tenemos 86606 artículos


In [2]:
id_to_art = {art["tweet_id"]:art for art in arts_v11}

In [4]:
import pandas as pd

ids = []


df_errores = pd.read_csv("TW sin datos.csv", encoding="latin_1")
            
print(f"Ids {len(df_errores)}")
print(f"Ids in v11: {sum(i in v11_ids for i in df_errores['ID_OT'])}")

ids = list(int(i) for i in df_errores["ID_OT"].values)

Ids 3541
Ids in v11: 3541


In [6]:
arts = Article.objects(tweet_id__in=ids)

art_ids = [art.tweet_id for art in arts]

print(f"En la bbdd: {arts.count()}")

En la bbdd: 3541


In [7]:
ids_set = set(art_ids)

In [8]:
missing_ids = [i for i in ids if i not in ids_set]

len(missing_ids)

0

No tenemos errores de ids perdidos! Perfecto

## Termino "COVID-19"

Parece que este término nos trae cualquier verdura. Veamos qué pasa acá

In [9]:
db.article.count_documents({"$text": {"$search": "covid"}, "tweet_id": {"$in": ids}})

26

In [10]:
db.article.count_documents({"$text": {"$search": "covid-19"}, "tweet_id": {"$in": ids}})

2173

Ok, hay un problema con este término. Será por el slash?

In [11]:
arts_with_19 = db.article.find(
    {"$text": {"$search": "19"}, "tweet_id": {"$in": ids}}, 
    {"tweet_id": 1}
)

ids_with_19 = [
    art["tweet_id"] for art in arts_with_19
]

In [12]:
len(ids_with_19)

2149

In [13]:
ids_with_19 = set(ids_with_19)
ids_without_19 = [i for i in ids if i not in ids_with_19]

In [14]:
len(ids_without_19)

1392

Nos quedan 1392. Veamos qué pasa acá

In [23]:
terminos = [
  "coronavirus", 
  "covid", 
  "cuarentena",
  "normalidad",
  "aislamiento",
  "padecimiento",
  "encierro",
  "fase",
  "infectados",
  "Wuhan",
  "distanciamiento",
  "fiebre",
  "síntomas",
  "dengue",
  "aedes",
  "mosquito",
  "descacharrar",
  "cacharro",
]

for termino in terminos:
    count = db.article.count_documents({
        "$text": {"$search": termino}, 
        "tweet_id": {"$in": ids_without_19}
    })
    
    print(f"Término {termino:<15} ----> {count:<4} resultados")

Término coronavirus     ----> 0    resultados
Término covid           ----> 24   resultados
Término cuarentena      ----> 0    resultados
Término normalidad      ----> 468  resultados
Término aislamiento     ----> 0    resultados
Término padecimiento    ----> 298  resultados
Término encierro        ----> 28   resultados
Término fase            ----> 16   resultados
Término infectados      ----> 8    resultados
Término Wuhan           ----> 0    resultados
Término distanciamiento ----> 616  resultados
Término fiebre          ----> 0    resultados
Término síntomas        ----> 0    resultados
Término dengue          ----> 2    resultados
Término aedes           ----> 0    resultados
Término mosquito        ----> 0    resultados
Término descacharrar    ----> 0    resultados
Término cacharro        ----> 0    resultados


Normalidad y padecimiento rankean

## Normalidad

Término candidato a irse...

In [24]:
for art in db.article.find({
    "$text": {"$search": "normalidad"}, 
    "tweet_id": {"$in": ids_without_19}
}):
    print("="*80 + "\n\n")
    print(f"{art['tweet_id']} -- {art['created_at']}")
    print(art["title"])
    print("\n\n")
    print(art["body"][:500])




1237017055162970112 -- 2020-03-09 14:07:03
Nació en un viaje y nunca fue a la escuela: la curiosa decisión de un joven que paró para llevar una vida “normal”



En Islas Canarias, Pampa Zapp, aún viviendo la vida nómada familiar

Es imposible que Pampa Zapp, de 17 años, pase desapercibido. No tiene que ver con su 1.80 metro de altura, ni su pequeño acento a la hora de hablar sino por su manera singular de relatar su increíble historia de vida. Todavía no alcanzó la mayoría de edad y ya pisó casi 90 países. De hecho nació en el camino de la aventura que comenzaron sus papás Herman y Candelaria en el 2000 a bordo de una reliquia: un automóvil Graham-Paige


1234648751328964609 -- 2020-03-03 01:16:15
El ciclo lectivo 2020 inició normalmente en la mayoría de las provincias



En la mayoría de las jurisdicciones educativas el comienzo del ciclo lectivo empezó en tiempo y forma. En Misiones las clases comenzarán el 9 de marzo, de acuerdo a su calendario, así como Mendoza arrancó el 26 de f

## Distanciamiento

In [31]:
for art in db.article.find({
    "$text": {"$search": "distanciamiento"}, 
    "tweet_id": {"$in": ids_without_19}
}):
    print("="*80 + "\n\n")
    print(f"{art['tweet_id']} -- {art['created_at']}")
    print(art["title"])
    print("\n\n")
    print(art["body"][:500])




1227328160498356232 -- 2020-02-11 20:26:50
Bustinduy, el opositor de la UTA: “Roberto Fernández es el responsable de convertir a los choferes de larga distancia en asesinos”



Bustinduy es el líder opositor dentro de la UTA (Foto Gentileza mundogremial.com)

“Roberto Fernández es el responsable de convertir a los choferes de larga distancia en asesinos”. La frase, que suena como un disparo al pecho del titular de la Unión Tranviarios Automotor (UTA), fue pronunciada ante Infobae por Miguel Ángel Bustinduy, el dirigente gremial que lidera el sector disidente del sindicato. El diálogo se desarrolla mientras parte del cuerpo de delegados, que responde a la agrupación Jua


1232835159449358336 -- 2020-02-27 01:09:41
“Nadie se muere sin sexo”: la relación a distancia del Tirri y Mimi, sin verse por dos años



Verónica Guerman / Teleshow

Ellos están juntos hace once años. Sin embargo, desde hace más de dos que no se ven. Aunque para muchos pueda resultar extraño, ellos se aman y eligen 

In [50]:
import datetime

query = {
    "user": {"$in": ["LANACION", "clarincom", "cronica", "infobae", "perfilcom"] }, 
    "created_at":{ "$gte": datetime.datetime(2020, 2, 10) },
    "$or": [
        {"$text": {"$search": "coronavirus covid cuarentena fase infectados Wuhan fiebre síntomas" }},
        {"$text": {"$search": "dengue aedes mosquito descacharrar cacharro"}},
    ]
}

db.article.count_documents(query)


OperationFailure: Failed to determine whether query system can provide a covered projection :: caused by :: Too many text expressions